In [176]:
import geopandas as gpd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from shapely.geometry import Polygon, mapping
import shapely
from shapely.ops import polygonize


In [177]:
user = "postgres"
password = "Ww4756"
host = "localhost"
port = 5432
database = "casestudy"
conn = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(conn)


In [178]:
miningArea = gpd.read_file('../caseData/maden_sinir/maden_sinirlari.dxf')

In [179]:
miningArea.head()

,Layer,PaperSpace,SubClasses,Linetype,EntityHandle,Text,geometry
0,0,NaN,AcDbEntity:AcDbLine,NaN,6E,NaN,"LINESTRING Z (465000.000 4399000.000 0.000, 46..."
1,0,NaN,AcDbEntity:AcDbLine,NaN,6F,NaN,"LINESTRING Z (463350.000 4399000.000 0.000, 46..."
2,0,NaN,AcDbEntity:AcDbLine,NaN,70,NaN,"LINESTRING Z (463350.000 4398083.000 0.000, 46..."
3,0,NaN,AcDbEntity:AcDbLine,NaN,71,NaN,"LINESTRING Z (498421.000 4411340.000 0.000, 49..."
4,0,NaN,AcDbEntity:AcDbLine,NaN,72,NaN,"LINESTRING Z (498589.000 4411617.000 0.000, 49..."


In [180]:
miningArea.loc[:,'id']= range(1,len(miningArea) + 1)
miningAreaV2= miningArea[['id','geometry']]
miningAreaV2.head()

,id,geometry
0,1,"LINESTRING Z (465000.000 4399000.000 0.000, 46..."
1,2,"LINESTRING Z (463350.000 4399000.000 0.000, 46..."
2,3,"LINESTRING Z (463350.000 4398083.000 0.000, 46..."
3,4,"LINESTRING Z (498421.000 4411340.000 0.000, 49..."
4,5,"LINESTRING Z (498589.000 4411617.000 0.000, 49..."


In [187]:
line_2d = shapely.force_2d(miningAreaV2.geometry)


In [182]:
polygons = gpd.GeoSeries(polygonize(line_2d.geometry))
polygons.crs = {'init':'epsg:23036'}
#polygons.crs= ("init"=="epsg:23036")


/Users/eyuptunahanunal/anaconda3/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [195]:
##polygons.geometry.to_file('../caseData/test/dataframe.shp')  
polygons##polygons.to_postgis(name="miningArea", con=engine, schema="public")

0     POLYGON ((463350.000 4399000.000, 465000.000 4...
1     POLYGON ((476000.000 4437000.000, 477000.000 4...
2     POLYGON ((506000.000 4414770.000, 506270.000 4...
3     POLYGON ((445161.000 4402868.002, 445349.000 4...
4     POLYGON ((407237.000 4394445.002, 407249.000 4...
5     POLYGON ((531511.000 4420667.002, 531556.000 4...
6     POLYGON ((520491.000 4461547.002, 520501.000 4...
7     POLYGON ((445453.000 4460792.002, 445445.000 4...
8     POLYGON ((437969.496 4427293.194, 437969.496 4...
9     POLYGON ((441026.455 4424291.816, 440859.711 4...
10    POLYGON ((510961.000 4456934.002, 510977.000 4...
11    POLYGON ((515498.000 4453516.002, 515528.000 4...
12    POLYGON ((445572.000 4403100.002, 445911.000 4...
13    POLYGON ((529408.000 4424469.002, 529265.000 4...
14    POLYGON ((468778.000 4443430.002, 468796.000 4...
15    POLYGON ((506000.000 4415255.000, 506500.000 4...
16    POLYGON ((383985.000 4443155.002, 384065.000 4...
17    POLYGON ((410943.000 4394711.002, 410908.0

In [197]:
for i, geom in enumerate(polygons):
    miningAreaV2.at[i, 'geometry'] = geom


In [199]:
filter_gdf=miningAreaV2[miningAreaV2['geometry'].notnull()]

In [202]:

filter_gdf.to_postgis(name="miningarea", con=engine, schema="public")